In [ ]:
import numpy as np
from tidygrad.tensor import Tensor
from tidygrad.utils.grad_check import grad_check

In [ ]:
def run_test_binary_elementwise(func, shape1, shape2=None, pos_only=False):
    """Test a binary elementwise function, like add, mul, etc"""
    shape2 = shape1 if shape2 is None else shape2
    if pos_only:
        a = Tensor(np.abs(np.random.randn(*shape1)) + 1e-8, name="a")
        b = Tensor(np.abs(np.random.randn(*shape2)) + 1e-8, name="b")
    else:
        a = Tensor(np.random.randn(*shape1), name="a")
        b = Tensor(np.random.randn(*shape2), name="b")

    t = func(inputs=None, params=(a, b))
    t.backward()
    grad_check(func=func, inputs=None, params=(a, b))

In [ ]:
def run_test_unary_elementwise(func, shape, pos_only=False):
    """Test a unary elementwise function, like exp, log, etc"""
    if pos_only:
        # Mostly for log(a) - it's positive only and is instable too close to zero.
        a = Tensor(np.abs(np.random.randn(*shape)) + 1e-3, name="a")
    else:
        a = Tensor(np.random.randn(*shape), name="a")

    t = func(inputs=None, params=(a, ))
    t.backward()
    grad_check(func=func, inputs=None, params=(a, ))

### Binary elementwise ops


In [ ]:
def add_func(inputs, params: tuple = ()):
    a, b = params
    loss = a.add(b, "t").sum("loss")
    return loss

run_test_binary_elementwise(add_func, (100, 100))

Max gradient difference for b: 0.0000%
Max gradient difference for a: 0.0000%


In [ ]:
def sub_func(inputs, params: tuple = ()):
    a, b = params
    loss = a.sub(b, "t").sum("loss")
    return loss

run_test_binary_elementwise(sub_func, (100, 100))

Max gradient difference for b: 0.0000%
Max gradient difference for a: 0.0000%


In [ ]:
def mul_func(inputs, params: tuple = ()):
    a, b = params
    loss = a.mul(b, "t").sum("loss")
    return loss

run_test_binary_elementwise(mul_func, (100, 100))

Max gradient difference for b: 0.0002%
Max gradient difference for a: 0.0006%


### Unary elementwise functions


In [ ]:
def log_func(inputs, params: tuple = ()):
    (a, ) = params

    loss = a.log("t").sum("loss")
    return loss

run_test_unary_elementwise(log_func, (100, 100), pos_only=True)

Max gradient difference for a: 0.2050%


In [ ]:
def exp_func(inputs, params: tuple = ()):
    (a, ) = params

    loss = a.exp("t").sum("loss")
    return loss

run_test_unary_elementwise(exp_func, (100, 100))

Max gradient difference for a: 0.0007%


In [ ]:
from tidygrad.functional import relu, sigmoid, softmax

In [ ]:
def sigmoid_text(inputs, params: tuple = ()):
    (a, ) = params
    t = sigmoid(a)
    return t.sum("loss")

run_test_unary_elementwise(sigmoid_text, (100, 100))

Max gradient difference for a: 0.0005%


In [ ]:
def relu_func(inputs, params: tuple = ()):
    (a, ) = params
    t = relu(a, "t")
    return t.sum("loss")

run_test_unary_elementwise(relu_func, (100, 100))

Max gradient difference for a: 0.0000%


In [ ]:
def softmax_func(inputs, params: tuple = ()):
    a, = params
    n_batch, n_classes = a.shape
    y = np.zeros(a.shape)
    np.random.seed(42)
    y[np.arange(n_batch), np.random.randint(0, n_classes, n_batch)] = 1
    y = Tensor(y, name="y")
    sm = softmax(a, "t")

    cross_entropy = y * sm.log() + (1-y) * (1 - sm).log()
    #
    return cross_entropy.sum("loss")

run_test_unary_elementwise(softmax_func, (1, 5))

Max gradient difference for a: 0.0005%


In [ ]:
def matmul_func(inputs, params: tuple[Tensor] = ()):
    a, b = params
    t = a.mmul(b, "t")
    return t.sum("loss")

def run_test_matmul(shape1, shape2):
    a = Tensor(np.random.randn(*shape1), name="a")
    b = Tensor(np.random.randn(*shape2), name="b")
    t = matmul_func(inputs=None, params=(a, b))
    t.backward()

    grad_check(func=matmul_func, inputs=None, params=(a, b))

run_test_matmul((10, 100), (100, 50))

Max gradient difference for b: 0.0000%
Max gradient difference for a: 0.0000%


### Broadcasting


In [ ]:
run_test_binary_elementwise(add_func, (2, 10, 1), (10, 100))

Max gradient difference for b: 0.0000%
Max gradient difference for a: 0.0000%


In [ ]:
run_test_matmul((2, 10, 100), (100, 10))

Max gradient difference for b: 0.0001%
Max gradient difference for a: 0.0000%


### Test loss functions


In [ ]:
# def lt_func(inputs, params: tuple = ()):
#     a, b = params
#     loss = (a < b).sum("loss")
#     return loss

# run_test_binary_elementwise(lt_func, (100, 100), (100, 100))

# a = Tensor(np.random.randn(100, 100), name="a")
# b = Tensor(np.random.randn(100, 100), name="b")

# t = lt_func(inputs=None, params=(a, b))
# t.backward()

In [ ]:
from tidygrad.functional import BCE_loss

In [ ]:
def bceloss_func(inputs, params: tuple = ()):
    y = inputs[0]
    x = params[0]

    loss = BCE_loss(x, y).sum("loss")
    return loss

x = Tensor(np.random.randn(100), name="x")
y = Tensor(np.random.randn(100), name="y")

t = bceloss_func(inputs=(y, ), params=(x, ))
t.backward()

grad_check(func=bceloss_func, inputs=(y, ), params=(x, ))

Max gradient difference for x: 0.0028%


### Test Dropout


In [ ]:
from tidygrad.functional import dropout

In [ ]:
def dropout_func(inputs, params: tuple = ()):
    p = params[0]

    np.random.seed(1337)
    t = dropout(p, 0.3, training=True)
    return t.sum("loss")

p = Tensor(np.random.randn(100), name="p")

t = dropout_func(inputs=None, params=(p, ))
t.backward()

grad_check(func=dropout_func, inputs=None, params=(p, ))

Max gradient difference for p: 0.0000%


In [ ]:
### Test sum and mean

In [ ]:
def sum_test(inputs, params: tuple = ()):
    a = params[0]
    t = a.sum("t")
    return t.sum("loss")

run_test_unary_elementwise(sum_test, (100, 100))

Max gradient difference for a: 0.0000%


In [ ]:
def mean_test(inputs, params: tuple = ()):
    a = params[0]
    t = a.mean("t")
    return t.sum("loss")

run_test_unary_elementwise(mean_test, (100, 100))

Max gradient difference for a: 0.0000%
